# Tom and Alex Violet Project 2
# April 13, 2025



In [18]:
import matplotlib.pyplot as plt
from CollabFilterOneVectorPerItem import CollabFilterOneVectorPerItem
from train_valid_test_loader import load_train_valid_test_datasets

import autograd
import autograd.numpy as ag_np
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import sklearn

from surprise import SVD,KNNBasic,KNNWithZScore,KNNWithMeans
from surprise import Dataset, Reader, accuracy
from surprise.model_selection import cross_validate
from surprise import model_selection

from surprise import SVD, SVDpp
from surprise import Dataset, Reader
from surprise.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8')

In [2]:
# ------------------------------------------------------


##LOADING DATA##
reader = Reader(
    line_format='user item rating', sep=',',
    rating_scale=(1, 5), skip_lines=1)

# train_data = Dataset.load_from_file(
#     'data_movie_lens_100k/ratings_all_development_set.csv', reader=reader)
# # print(train_set[:5])
# train_set = train_data.build_full_trainset()
# raw_ratings = train_set.build_testset()


##MAKING MERGE DATA FRAME
# df = pd.read_csv('data_movie_lens_100k/ratings_all_development_set.csv')
# user_df = pd.read_csv('data_movie_lens_100k/user_info.csv')
# movie_df = pd.read_csv('data_movie_lens_100k/movie_info.csv')
# merged_df = pd.merge(df, user_df, on='user_id', how='inner')
# merged_df = merged_df.drop(columns=['orig_user_id'])
# merged_movie_df = pd.merge(merged_df, movie_df, on='item_id', how='inner')



# user_df = pd.read_csv('data_movie_lens_100k/user_info.csv')'
# print(merged_movie_df[:100])
# data = Dataset.load_from_df(df[['user_id', 'item_id', 'rating']], reader)



DATA_PATH = 'data_movie_lens_100k/' # TODO fixme: Path to where dataset csv files live on your system
if not os.path.exists(os.path.join(DATA_PATH, 'select_movies.csv')):
    try:
        DATA_PATH = os.path.join(os.environ.get("HOME", ""),
                    'courses/cs135-25s-staffonly/proj_src/projB/data_movie_lens_100k/')
        assert os.path.exists(os.path.join(DATA_PATH, 'select_movies.csv'))
    except AssertionError:
        print("Please store path to movie_lens_100k dataset in DATA_PATH")
assert os.path.exists(os.path.join(DATA_PATH, 'select_movies.csv'))


df_masked = pd.read_csv("data_movie_lens_100k/ratings_masked_leaderboard_set.csv")

df_masked['user_id'] = df_masked['user_id'].astype(str)
df_masked['item_id'] = df_masked['item_id'].astype(str)
df_masked['rating'] = 0


# Create a list of (uid, iid) pairs
predict_pairs = list(zip(df_masked['user_id'], df_masked['item_id'],df_masked['rating']))


dev_set = Dataset.load_from_file(
   os.path.join(DATA_PATH, 'ratings_all_development_set.csv'), reader=reader)
dev_set_for_fit = dev_set.build_full_trainset()
dev_set_for_predict = dev_set_for_fit.build_testset()
dev_set_for_fit.global_mean

print("Global Mean: ", dev_set_for_fit.global_mean)


Global Mean:  3.529480398257623


In [ ]:
##TRAINING##---------------------------------------



# param_grid = {
#     # "n_factors": [20, 50, 100],        # Latent dimensions (embedding size)
#     # "lr_all": [0.002, 0.005, 0.01],    # Learning rate for all parameters
#     # "reg_all": [0.02, 0.05, 0.1],      # Regularization term for all parameters
#     # "n_epochs": [10, 20, 30]
#     "n_factors": [100],        # Latent dimensions (embedding size)
#     "lr_all": [0.01],    # Learning rate for all parameters
#     "reg_all": [0.1],      # Regularization term for all parameters
#     "n_epochs": [30]          # Number of SGD iterations
# }
# model = model_selection.search.RandomizedSearchCV(SVDpp, param_grid, n_iter=1, measures=['mae'], refit=True, n_jobs=-1)
# model.fit(dev_set) 
# print("Lowest MAE: ", model.best_score['mae'])
# print("Best params:", model.best_params['mae'])
# best_model = model.best_estimator['mae']
# best_model.fit(dev_set_for_fit)

##TRAIN AND FIT
# best_model = SVDpp(n_factors=100, lr_all=0.01, reg_all=0.1, n_epochs=30)
# best_model.fit(dev_set_for_fit)


In [62]:
param_grid = {
    'k': [30,40,50,60,70],
    'sim_options': {
        'name': ['cosine', 'pearson', 'pearson_baseline'],
        'user_based': [True, False]  # True = user-user, False = item-item
    },
    'min_k': [1, 3, 5,10,50],
    'verbose': [False]
}
model = model_selection.search.RandomizedSearchCV(KNNWithMeans, param_grid, n_iter=30, measures=['mae'], refit=True, n_jobs=-1)
model.fit(dev_set) 
best_model = model.best_estimator['mae']
best_model.fit(dev_set_for_fit)

predictions = [best_model.predict(uid, iid) for (uid, iid, _) in predict_pairs]
for i in range(25):
    print(predictions[i][2],predictions[i][3])

None 3.5667103434573733
None 4.134996832585336
None 3.7888666935126736
None 3.6720992658741642
None 3.8968649321958186
None 4.3203326230059425
None 2.175935469500353
None 3.8692354968993916
None 3.7887285572112375
None 3.4842793659510547
None 3.7855537153118766
None 3.7903893615687245
None 3.336980700997417
None 4.86406198051976
None 3.401681745386318
None 2.8788290563805177
None 2.729279484504305
None 2.9380884287811284
None 3.8273759218275765
None 3.6171707478948694
None 4.301944711322053
None 3.3799950999396975
None 2.944219982144821
None 2.7126847038918402
None 2.4807780523026812


In [63]:
##KNNBasic
print(model.best_score)
print(model.best_params)

print(model.cv_results)


{'mae': 0.7263855473175452}
{'mae': {'k': 40, 'sim_options': {'name': 'pearson_baseline', 'user_based': False}, 'min_k': 3, 'verbose': False}}
{'split0_test_mae': array([0.74791444, 0.76376255, 0.74457526, 0.73805637, 0.74718819,
       0.74667795, 0.73695238, 0.74589699, 0.74834338, 0.74989378,
       0.74607514, 0.83203677, 0.76367024, 0.74921287, 0.74599713,
       0.74738594, 0.73681715, 0.7297193 , 0.83203677, 0.77777786,
       0.82087537, 0.74581448, 0.75708139, 0.7452644 , 0.82087537,
       0.7773189 , 0.74676641, 0.76704444, 0.7563249 , 0.73699752]), 'split1_test_mae': array([0.74078509, 0.75907065, 0.74557248, 0.73781048, 0.74123383,
       0.74747088, 0.73841388, 0.73915558, 0.74849052, 0.74378835,
       0.74577998, 0.83580436, 0.75891014, 0.7515225 , 0.73946133,
       0.74758909, 0.73788417, 0.72558219, 0.83580436, 0.77766849,
       0.81594008, 0.74549879, 0.75916391, 0.73837662, 0.81594008,
       0.77757114, 0.74741821, 0.76090882, 0.75550332, 0.73762723]), 'split2_te

In [19]:
##KNNWITHZSCORE
print(model.best_score)

{'mae': 0.728028974844136}


In [21]:
##KNNWITHMEAN
print(model.best_score)

{'mae': 0.7258303388876878}


In [ ]:
ratings_only = [pred[3] for pred in predictions]  # extract just the ratings
ratings_only = np.array(ratings_only, dtype=float)
np.savetxt("predicted_ratings_leaderboard.txt", ratings_only, fmt="%.6f")
